# **Alumno**: Álvaro Manuel Aparicio Morales

## **Módulo**: Inteligencia Artificial Cuántica

I CERTIFICADO DE EXTENSIÓN UNIVERSITARIA EN COMPUTACIÓN CUÁNTICA

(2024-25)


**Exercise on the Quantum Classifier**

**Evaluation of the Machine Learning Part**

**Part 3: Earthquake Classifier**

Natural phenomena can be catastrophic. It is therefore important to develop prevention systems to classify high-risk areas. As an example, several types of earthquakes exist: volcanic, human, etc. Regardless of their type, let us assume that an earthquake depends on the frequency of tectonic plate events, as well as volcanic activity and mining facilities. We want to develop a quantum variational classifier that takes these characteristics as input to know whether an area is at risk of earthquakes or not.
1. Construct the included quantum variational classifier, U_{Φ(X_{i})} and U_{Φ(W(Φ)} using the transformation σ_{3} and a circuit depth Z equal to 1 and d = 3 for the transformation U_{Φ(X_{i})} and a depth of 1 for U_{Φ(W(Φ)}?
2.   Run the circuit with an IBM simulator, indicating the output of the classifier C computes Ŷ when the input data are 40, 3, and 10 and Θ = Π/3? The output refers to class 0 (earthquake is unlikely to occur) or 1 (earthquake is unlikely to occur).
You have to create the circuit using “IBM Quantum Composer” and deliver the OPENQASM code generated by the composer. Also, as a second option, you can use QISKIT to build the circuit. In both options, the deliverable would be a link to your Google Colab notebook.


In [1]:
!pip install qiskit[visualization]
!pip install qiskit
!pip install qiskit-ibm-runtime
!pip install qiskit-aer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 4.2 MB/s eta 0:00:00
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136816 sha256=787b07dd43a791dc782ac9c76b374fa882375013bdfb4d3551f400db78d5e51a
  Stored in directory: /root/.cache/pip/wheels/d3/31/8b/e09b0386afd80cfc556c00408c9aeea5c35c4d484a9c762fd5
Successfully built pylatexenc
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from numpy import pi
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from functools import reduce
from itertools import combinations

In [21]:
def generate_combinations(n):
  result = []
  for r in range(1, n+1):
    result.extend([list(comb) for comb in combinations(range(1, n + 1), r)])
    # Devuelve si n = 3, como es el caso, [[1], [2], [3], [1,2] [1,3], [2,3], [1,2,3]]
  return result

def get_pair_combinations(n):
  all_combinations = generate_combinations(n)
  result = [c for c in all_combinations if len(c) == 2]
  return result

def dataMappingAnsatz(qc, vector):
  # Aplicar una walls Hadamard
  for i in range(qc.num_qubits):
    qc.h(i)
  # Generar combinaciones e iterar para generar el sumatorio
  combinations = generate_combinations(len(vector))
  sum = 0
  for elem in combinations:
    item = 0
    if len(elem) == 1:
      qc.rz(2*vector[elem[0]-1], elem[0]-1) # Debido a que las combinaciones empiezan en 1, hay que restarle 1 al elemento de la combinacion.
    elif len(elem) == 2: # Qiskit limit of RZ
      qc.rzz(2 * (pi - vector[(elem[0]-1)]) * (pi - vector[(elem[1]-1)]),(elem[0]-1), (elem[1]-1))
    elif len(elem) == 3:
      # RZZ
      angle = 2 * (pi - vector[(elem[0]-1)]) * (pi - vector[(elem[1]-1)]) * (pi - vector[(elem[2]-1)])
      qc.cx((elem[0]-1), (elem[1]-1))
      qc.cx ((elem[1]-1), (elem[2]-1))
      qc.barrier()
      qc.rz(angle, (elem[2]-1))
      qc.barrier()
      qc.cx ((elem[1]-1), (elem[2]-1))
      qc.cx((elem[0]-1), (elem[1]-1))
      qc.barrier()
    else:
      pass
  return qc

def classificationAnsatz(qc, theta, layers):
  for l in range(layers):
    for q in range(qc.num_qubits):
      qc.ry(theta, q)
      qc.rz(theta, q)
    pair_combinations = get_pair_combinations(qc.num_qubits)
    for comb in pair_combinations:
      qc.cz(comb[0]-1, comb[1]-1)
  return qc

def prediction(bits):
  return reduce(lambda x, y: x ^ y, [int(bit) for bit in bits])



In [25]:

# Empezamos generando una lista de valores

earthquake_data = [[40,3,10]] # [[frequency_volcanic_plate_events, volcanic_activity, mining_facilities], [...],  ..., [...] ]
# Vamos a suponer que los datos vienen correctos y no hay que hacer ningún tratamiento previo
dimension = len(earthquake_data[0])
# Defining Variables
theta = pi/3
layers = 1
# Para cada tupla de entrada, hay que hacer los siguientes pasos
for data_input in earthquake_data:
  # Inicializamos el circuito
  qc = QuantumCircuit(dimension)
  # Añadimos las fases
    # Transformar el dato clásico a cuántico (data mapping ansatz)
  qc = dataMappingAnsatz(qc, data_input)
    # Aplicar la clasificación
  qc = classificationAnsatz(qc, theta, layers)
    # Aplicar medición de la predicción
  qc.measure_all()
  print(qc)

  # Ejecutamos
  backend = AerSimulator()
  task = backend.run(qc, shots=1)
  output = task.result()
  measure_data = list(output.get_counts().keys())[0]
  earthquake = prediction(measure_data)
  print("Result : ", earthquake)
  if earthquake == 0:
      print("PANIC !! An earthquake is comming !! ")
  else:
      print("Don't worry, be happy")

Combinaciones
[[1], [2], [3], [1, 2], [1, 3], [2, 3], [1, 2, 3]]
Combinaciones
[[1], [2], [3], [1, 2], [1, 3], [2, 3], [1, 2, 3]]
        ┌───┐┌────────┐                                                    ░ »
   q_0: ┤ H ├┤ Rz(80) ├─■─────────────■───────────────────────────■────────░─»
        ├───┤├───────┬┘ │ZZ(-10.438)  │                         ┌─┴─┐      ░ »
   q_1: ┤ H ├┤ Rz(6) ├──■─────────────┼────────────■────────────┤ X ├──■───░─»
        ├───┤├───────┴┐               │ZZ(505.58)  │ZZ(-1.9422) └───┘┌─┴─┐ ░ »
   q_2: ┤ H ├┤ Rz(20) ├───────────────■────────────■─────────────────┤ X ├─░─»
        └───┘└────────┘                                              └───┘ ░ »
meas: 3/═════════════════════════════════════════════════════════════════════»
                                                                             »
«                       ░            ░ ┌─────────┐┌─────────┐          ░ ┌─┐   »
«   q_0: ───────────────░────────■───░─┤ Ry(π/3) ├┤ Rz(π/3) ├─■──■─────░─┤M├──